In [1]:
import importlib

# Logging
import logging
logging.basicConfig(level=logging.ERROR, filename='test_quiz.log')
logger = logging.getLogger('ollama')

In [2]:
# Setup sentence encoder
from sentence_transformers import SentenceTransformer
st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
embedder = st5_model.encode

In [3]:
# Set up LLM
import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)
path = os.path.abspath('..')
if path not in sys.path:
    sys.path.insert(0, path)

from concordia.language_model import ollama_model
model = ollama_model.OllamaLanguageModel(
    model_name='llama2:13b'
    # model_name='mixtral'
)

In [4]:
import custom_components.build_quiz_agents as build_agents
importlib.reload(build_agents)

SCENARIO = "mcat"
YEAR = 2024

m_names = build_agents.generate_names(model, 'men')
f_names = build_agents.generate_names(model, 'women')
names = [m_names, f_names]

print(len(names[0]), len(names[1]))

33 28


In [5]:
importlib.reload(build_agents)

for i in range(1):
    clock = build_agents.build_clock(YEAR)
    agent = build_agents.build_random_agent(model, names, clock, SCENARIO, i)

    filepath = f"./agents/{YEAR}/{SCENARIO}/{agent.agent_id}.pkl"
    
    agent.save_agent(filepath)


Creating backstory for Aidan using the following information:
Age: 56
Current year: 2024
Traits: general knowledge: medium; medical knowledge: very low
Gender: male



In [6]:
from concordia.utils import measurements as measurements_lib
measurements = measurements_lib.Measurements()
import examples.custom_components.quiz_metric as qm
importlib.reload(qm)

all_results = []
run_number = []
N_RUNS = 5

for i in range(1):

    for j in range(N_RUNS):

        agent = build_agents.QuizAgent.load_agent(model, clock, f'./agents/{YEAR}/{SCENARIO}/{i}.pkl')

        if SCENARIO == 'personality':
            test_context = (
                '\n'
                f'{agent.name} is taking a personality quiz just for fun.'
                f'Here are a number of personality traits that may or may not apply to {agent.name}. '
                f'Please indicate the extent to which {agent.name} would agree or disagree with that statement. '
                f'{agent.name} should rate the extent to which the pair of traits applies to '
                f'{"her" if agent.gender == "female" else "him"}, even if one characteristic applies more strongly than the other.'
                '\n'
            )
            file_path = './quizzes/big_five_questions.json'
        if SCENARIO == 'trivia':
            test_context = (
                '\n'
                f"{agent.name} is taking a pop culture trivia quiz just for fun."
                f"If {agent.name} doesn't know the answer, {'she' if agent.gender == 'female' else 'he'} will still guess "
                f"and begin {'her' if agent.gender == 'female' else 'his'} answer with a single choice. "
                f"How would {agent.name} answer the following question?"
                '\n'
            )
            file_path = './quizzes/trivia_questions.json'
        if SCENARIO == "mcat":
            test_context = (
                '\n'
                f"{agent.name} is taking an MCAT preparatory quiz just for fun."
                f"If {agent.name} doesn't know the answer, {'she' if agent.gender == 'female' else 'he'} will still guess "
                f"and begin {'her' if agent.gender == 'female' else 'his'} answer with a single choice. "
                f"How would {agent.name} answer the following question?"
                '\n'
            )
            file_path = './quizzes/mcat_questions.json'

        context = agent.backstory + test_context

        quiz_metric = qm.QuizMetric(model, agent, file_path, measurements = measurements, verbose = True)
        results = quiz_metric.observe(context, return_data=True)

        run_number.append(j)
        all_results.append(results)



Answer: d
Sample:  Based on the information provided, Aidan would answer the question with choice (d) CAR T-cell therapy.
Question: What recent development in cancer treatment involves modifying a patient's own T cells to attack cancer cells?
Aidan's answer: CAR T-cell therapy
Correct answer: CAR T-cell therapy

Answer: d
Sample:  Based on the information provided, Aidan would answer the question as follows:

(d) Rivaroxaban.
Question: Which of the following is a novel oral anticoagulant (NOAC) introduced in the last 20 years?
Aidan's answer: Rivaroxaban
Correct answer: Rivaroxaban

Answer: b
Sample: Based on the information provided, Aidan would answer the question as follows:

(b) Pre-exposure Prophylaxis
Question: In the context of HIV treatment, what does the term 'PrEP' stand for?
Aidan's answer: Pre-exposure Prophylaxis
Correct answer: Pre-exposure Prophylaxis

Answer: d
Sample:  Based on the information provided, Aidan would likely answer the question by guessing and choosing on

In [7]:
import csv

with open(f'./data/llama2_13b_{SCENARIO}_{YEAR}_agents.csv', 'w') as newfile:
    writer = csv.writer(newfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['agent_id', 'agent_name', 'traits', 'question', 'answer', 'correct', 'run'])
    for i in range(len(all_results)):
        run = run_number[i]
        results = all_results[i]
        for result in results:
            row = [item.strip() for item in result.split(",")]
            row.append(run)
            writer.writerow(row)